In [1]:
from transformers import OPTForCausalLM, AutoTokenizer
import torch
from torch import nn 

import torch
import torch.nn as nn
from transformers import OPTForCausalLM

import numpy as np
from numpy.lib.format import open_memmap

from threading import Thread
from queue import Queue 

import functools 

/llama_ft/miniconda3/envs/trt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
checkpoint = 'facebook/opt-125m'

In [3]:
def find_module_list(module: nn.Module):
    def _find_module_list(module: nn.Module):
        if isinstance(module, nn.ModuleList):
            yield module
        else:
            for child in module.children():
                yield from _find_module_list(child)
    
    g = _find_module_list(module)
    try:
        return next(iter(g))
    except:
        raise ValueError(f'{module.__class__.__name__} does not have a nn.ModuleList structure')

import torch
from accelerate.utils import honor_type
from typing import Mapping

def get_info(obj):
    if isinstance(obj, (tuple, list)):
        # return honor_type(obj, (get_info(o) for o in obj))
        return f"{len(obj)}x {get_info(obj[0])}"
    elif isinstance(obj, Mapping):
        return type(obj)({k: get_info(v) for k, v in obj.items()})
    elif isinstance(obj, (torch.Tensor)):
        return f"{obj.__class__.__name__}(shape={tuple(obj.size())}, dtype={obj.dtype}, device={obj.device})"
    # elif isinstance(obj, (MixTensor)):
    #     return f"{obj.__class__.__name__}(shape={tuple(obj.size())}, dtype={obj.dtype}, percents={obj.percents})"
    # elif isinstance(obj, (BatchListTensor)):
    #     return f"{obj.__class__.__name__}(shape={tuple(obj.size())}, dtype={obj.dtype}, has_mix={obj.has_mix})"
    elif isinstance(obj, (int, bool, type(None))):
        return f"{obj}"
    else:
        # logger.warning(f"inputs: {obj} of type '{type(obj)}' is not implemented.")
        return f"{obj.__class__.__name__}: {obj}"


class Model:
    """workload balance"""
    def __init__(self, hf_model, ) -> None:
        self.hf_model = hf_model 
        self.layers = self.get_layers()

    def get_layers(self):
        if isinstance(self.hf_model, (OPTForCausalLM, )):
            return self.hf_model.model.decoder.layers 
        else:
            return find_module_list(self.hf_model)
    
    def override_layer_forward(self, i: int):
        layer = self.layers[i]
        old_forward = layer.forward

        @functools.wraps(old_forward)
        def new_forward(*args, **kwargs):
            print(f'{i = }, \n\t{get_info(args) = }, \n\t{get_info(kwargs) = }')

            if isinstance(self.hf_model, (OPTForCausalLM, )):
                actv_recomp = args[0] # b,1,h / bzh
                kv_cache = kwargs.get('past_key_value') # b,n_kv_heads,s_cache,h_kv    x2
                attn_mask = kwargs.get('attention_mask') # b,1,1,s_all  (bsz, 1, tgt_len, src_len)

            # wlb formatted args, kwargs to formats for the original forward of hf model
            args_for_old = args
            kwargs_for_old = kwargs
            output = old_forward(*args_for_old, **kwargs_for_old) # h'=(b,z,h), kv=(b,n,s_all,h) x2
            
            print(f'{i = }, \n\t{get_info(output) = }')
            
            return output
        
        layer.forward = new_forward
        return layer

    def build(self):
        for i, _ in enumerate(self.layers):
            self.override_layer_forward(i)
        return self.hf_model 

hf_model= OPTForCausalLM.from_pretrained(checkpoint)
x_model = Model(hf_model).build()

/llama_ft/miniconda3/envs/trt/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
num_prompts = 16
prompts = None
checkpoint = checkpoint
prompt_len = 50
compute_device = 0
gen_len = 20
model = x_model.to(compute_device) # 

# test
if True:
    if prompts is None:  # get default prompts
        prompts = [
            "for i in range(10): ",
            "Who are you? Are you conscious?",
            "Where is Deutschland?",
            "How is Huawei Mate 60 Pro?",
        ]
    prompts = (
        prompts * (num_prompts // len(prompts))
        + prompts[: (num_prompts % len(prompts))]
    )

    # tokenizer
    tokenizer = AutoTokenizer.from_pretrained(checkpoint) # , padding_side="left"
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token  # eos padding

    # inputs
    inputs = tokenizer(
        prompts,
        padding="max_length",
        max_length=prompt_len,
        return_tensors="pt",
        # padding=True,
    ).to(compute_device)

    # generate
    generate_ids = model.generate(
        inputs.input_ids,
        max_new_tokens=gen_len,  # max_lengths
        # num_beams=2, #
        # num_beam_groups=2, #
        # diversity_penalty=0.1, #
        # do_sample=True, #
    )

    # outputs
    output_texts = tokenizer.batch_decode(
        generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    print(output_texts)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


get_info(model_inputs) = {'input_ids': 'Tensor(shape=(16, 50), dtype=torch.int64, device=cuda:0)', 'past_key_values': 'None', 'use_cache': 'True', 'attention_mask': 'Tensor(shape=(16, 50), dtype=torch.int64, device=cuda:0)'}
i = 0, 
	get_info(args) = ('Tensor(shape=(16, 50, 768), dtype=torch.float32, device=cuda:0)',), 
	get_info(kwargs) = {'attention_mask': 'Tensor(shape=(16, 1, 50, 50), dtype=torch.float32, device=cuda:0)', 'layer_head_mask': 'None', 'past_key_value': 'None', 'output_attentions': 'False', 'use_cache': 'True'}
i = 0, 
	get_info(output) = ('Tensor(shape=(16, 50, 768), dtype=torch.float32, device=cuda:0)', ('Tensor(shape=(16, 12, 50, 64), dtype=torch.float32, device=cuda:0)', 'Tensor(shape=(16, 12, 50, 64), dtype=torch.float32, device=cuda:0)'))
i = 1, 
	get_info(args) = ('Tensor(shape=(16, 50, 768), dtype=torch.float32, device=cuda:0)',), 
	get_info(kwargs) = {'attention_mask': 'Tensor(shape=(16, 1, 50, 50), dtype=torch.float32, device=cuda:0)', 'layer_head_mask': 'Non